In [1]:
%load_ext autoreload
import numpy as np

%autoreload 2
from zcu_tools.datasaver import load_data
import zcu_tools.analysis.notebook.tools as znt
import zcu_tools.analysis.notebook.dispersive as znd

In [2]:
qub_name = "Test096"

In [ ]:
loadpath = f"../../result/{qub_name}/params.json"
_, params, mA_c, period, allows = znt.load_result(loadpath)

if "r_f" in allows:
    r_f = allows["r_f"]
    print(f"r_f = {r_f}")

# Plot with Onetone

In [4]:
onetone_path = "../../Database/S001/2025/03/Data_0327/Test096_OneTone_flux_020.hdf5"

signals, sp_fpts, sp_mAs = load_data(
    onetone_path, server_ip="005-writeboard", port=4999
)
sp_mAs, sp_fpts, signals = znt.format_rawdata(sp_mAs, sp_fpts, signals)
signals = signals.T  # (sp_mAs, sp_fpts)

sp_flxs = znt.mA2flx(sp_mAs, mA_c, period)

In [5]:
# r_f = 7.196
best_g = 0.1

In [ ]:
best_g = znd.auto_fit_dispersive(
    params, r_f, sp_flxs, sp_fpts, signals, g_bound=(0.0, 0.2), g_init=best_g
)
best_g

In [ ]:
%matplotlib widget
finish_fn = znd.search_proper_g(
    params, r_f, sp_flxs, sp_fpts, signals, g_bound=(0.04, 0.15), g_init=best_g
)

In [ ]:
best_g = finish_fn()
best_g

In [9]:
flxs = np.linspace(sp_flxs.min(), sp_flxs.max(), 501)
mAs = znt.flx2mA(flxs, mA_c, period)

In [ ]:
rf_0, rf_1 = znd.calculate_dispersive(params, r_f, best_g, flxs)

In [ ]:
fig = znd.plot_dispersive_with_onetone(
    r_f, best_g, mAs, flxs, rf_0, rf_1, sp_mAs, sp_flxs, sp_fpts, signals
)
fig.show()

In [ ]:
fig.write_html(f"../../result/{qub_name}/web/dispersive.html", include_plotlyjs="cdn")
fig.write_image(
    f"../../result/{qub_name}/image/dispersive.png", format="png", width=800, height=400
)